Modélisation du problème de "construction d'immeubles"

Les données:
* m le nombre d'immeubles
* n le nombre de clients
* pour i allant de 1 à n
* p_i nombres d'étages voulant être occupés par le client i
* g_i le gain rapporté par le client i
* K la taille max d'un immeuble (somme des demandes d'étages)
* pour j allant de 0 à K
* P_j le prix de construction de chaque étage ~~le prix de l'immeuble s'il est de taille j~~

In [1]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;


In [2]:
#PLNE du problème général
#version avec toutes les données en argument

function skyline_problem(buildings_number,clients_number
        ,clients_floors,client_gains,building_max_size,floor_prices)


    # Création d'un modèle. Ce modèle fera l'interface avec le solveur GLPK
    model = Model(with_optimizer(GLPK.Optimizer))

    # Création des variables :
    
    # la demande du client i est attribué à l'immeuble l: 
    # variable binaire A[i,l] = 1 si oui, 0 sinon
    @variable(model,A[1:clients_number,1:buildings_number],Bin)
    
    # l'immeuble l a k étages
    # variable binaire B[k,l] = 1 si l'étage est occupé, 0 sinon
    @variable(model,B[1:building_max_size,1:buildings_number],Bin)
    #variable entière qui contiendra la taille des immeubles en étage
    @variable(model,0<=B_Bis[1:buildings_number]<=building_max_size,Int)
    # les deux types de variables sont nécessaires:
    #-les variables binaires permettent d'écrire la fonction objective, 
    # (on ne peut pas utilisé B comme indice d'un array pour avoir le prix d'un étage)
    #-B_Bis fait le lien entre B et la taille des étages demandés par les clients
    
    # Fonction objectif
    @objective(model, Max, 
        sum(clients_gains[i].*A[i,l] for i in 1:clients_number for l in 1:buildings_number) #somme des gains
        -
        sum(floor_prices[k].*B[k,l] for k in 1:building_max_size for l in 1:buildings_number)) #somme des prix des immeubles

    #Création des contraintes

    #contrainte : chaque client ne peut être attribué qu'à au plus 1 immeuble
    for i in 1:clients_number
        @constraint(model,sum(A[i,:]) <= 1)
    end
    
    #contrainte sur la taille des immeubles
    
    for l in 1:buildings_number
        #assure le lien entre B_Bis la taille des immeubles et B l'attribution des étages
        @constraint(model,B_Bis[l]==sum(B[k,l] for k in 1:building_max_size))
        #assure qu'on ne peut pas construire un étage supérieur si celui du dessous n'est pas construit
        for k in 1:building_max_size-1
            @constraint(model,B[k,l]>=B[k+1,l])
        end
        #assure le lien entre B_Bis la taille des immeubles et les demandes des clients
        @constraint(model,B_Bis[l]==sum(clients_floors[i].*A[i,l] for i in 1:clients_number))
    end

    #Affichage du modèle
    print(model)

    #Résolution du problème d'optimisation linéaire m par le solveur GLPK
    optimize!(model)

    status = termination_status(model)

    if status == INFEASIBLE
        println("Le problème n'est pas réalisable")
    elseif status == UNBOUNDED
        println("Le problème est non borné")
    elseif status == OPTIMAL
        println("Optimum = ", JuMP.objective_value(model))
        println("Solution optimale :")
        println("A = ",JuMP.value.(A))
        println("B = ",JuMP.value.(B))
        println("B_Bis = ",JuMP.value.(B_Bis))
        
        println("Affichage :")
        for l in 1:buildings_number
            println("\t-L'immeuble ",l," est de taille ",JuMP.value(B_Bis[l])," étages.")
            cout = sum(floor_prices[k].*JuMP.value(B[k,l]) for k in 1:building_max_size)
            println("\tIl coute ",cout,"€ à construire.")
            for i in 1:clients_number 
                if JuMP.value(A[i,l])==1
                    println("\t\tClient ",i," attribué à l'immeuble ",l)
                end
            end
            gain = sum(clients_gains[i].*JuMP.value(A[i,l]) for i in 1:clients_number)
            println("\tLes Clients rapportent ",gain,"€.")
            println("\tL'immeuble ",l," rapporte ",gain-cout," €.")    
            println()
        end
        
        println("Le projet nous rapporte ", JuMP.objective_value(model),"€.")
        println("dessin de la solution:")
        println()
        for l in 1:buildings_number
            print(l,"\t")
        end
        println()
        for k in 1:building_max_size
            for l in 1:buildings_number
                if JuMP.value(B[building_max_size-k+1,l])==0
                    print(".\t")
                else
                    print("M\t")
                end
            end
            println()
        end
    end
end

skyline_problem (generic function with 1 method)

In [3]:
#création d'une instance test du jeu:
#nombre d'immeubles
buildings_number = 3
#nombre de clients
clients_number = 4
#nombre d'étages pour chaque demande des clients
clients_floors = [1 1 3 4]
#gains pour chaque demande des clients
clients_gains = [5 8 100 300]
#taille maximal d'un immeuble = somme des étages de toutes les demandes
building_max_size = sum(clients_floors)
#prix de construction de chaque étage d'un immeuble
#pourra être créé via une fonction
floor_prices = [10 20 40 80 160 320 640 1280 2560] 

skyline_problem(buildings_number,clients_number
        ,clients_floors,clients_gains,building_max_size,floor_prices)

Max 5 A[1,1] + 5 A[1,2] + 5 A[1,3] + 8 A[2,1] + 8 A[2,2] + 8 A[2,3] + 100 A[3,1] + 100 A[3,2] + 100 A[3,3] + 300 A[4,1] + 300 A[4,2] + 300 A[4,3] - 10 B[1,1] - 10 B[1,2] - 10 B[1,3] - 20 B[2,1] - 20 B[2,2] - 20 B[2,3] - 40 B[3,1] - 40 B[3,2] - 40 B[3,3] - 80 B[4,1] - 80 B[4,2] - 80 B[4,3] - 160 B[5,1] - 160 B[5,2] - 160 B[5,3] - 320 B[6,1] - 320 B[6,2] - 320 B[6,3] - 640 B[7,1] - 640 B[7,2] - 640 B[7,3] - 1280 B[8,1] - 1280 B[8,2] - 1280 B[8,3] - 2560 B[9,1] - 2560 B[9,2] - 2560 B[9,3]
Subject to
 A[1,1] binary
 A[1,2] binary
 A[1,3] binary
 A[2,1] binary
 A[2,2] binary
 A[2,3] binary
 A[3,1] binary
 A[3,2] binary
 A[3,3] binary
 A[4,1] binary
 A[4,2] binary
 A[4,3] binary
 B[1,1] binary
 B[1,2] binary
 B[1,3] binary
 B[2,1] binary
 B[2,2] binary
 B[2,3] binary
 B[3,1] binary
 B[3,2] binary
 B[3,3] binary
 B[4,1] binary
 B[4,2] binary
 B[4,3] binary
 B[5,1] binary
 B[5,2] binary
 B[5,3] binary
 B[6,1] binary
 B[6,2] binary
 B[6,3] binary
 B[7,1] binary
 B[7,2] binary
 B[7,3] binary
 B[

In [ ]:
#algorithme de programmation dynamique pour m=1

#remarque: items commencé à 1 et non à 0
function skyline_problem_dynamic_resolution(clients_number,
        clients_floors,client_gains,building_max_size)
    V = zeros(Int64,building_max_size+1,clients_number)

    for k in 1:building_max_size+1 
        if  k > clients_floors[1]
            V[k,1] = clients_gains[1]
        end
    end
    for k in 1:building_max_size+1     
        for i in 2:clients_number
            if k <= clients_floors[i]
                V[k,i] = V[k,i-1]
            else
                V[k,i] = max(V[k,i-1],clients_gains[i]+V[k-clients_floors[i],i-1])
            end
        end
    end
    print("\t")
    for i in 1:clients_number
            print(i,"\t")
    end
    println()
    for k in 1:building_max_size+1
        print(k-1,"\t")
        for i in 1:clients_number
            print(V[k,i],"\t")
        end
        println()
    end
    println("PATH:")
    k = building_max_size+1
    for invi in 1:clients_number-1
        i = clients_number - invi +1
        println("(k,i,V[k,i]) = (",k-1,",",i,",",V[k,i],")")
        if V[k,i] != V[k,i-1]
            k = k - clients_floors[i]
        end
    end
    println("(k,i,V[k,i]) = (",k-1,",",1,",",V[k,1],")")
end




In [ ]:

#resolution d'un sac à dos, exo3 TD7 d'OC pour vérification

clients_number = 5
clients_floors = [7 6 14 7 9]
clients_gains = [1 2 5 6 11]
building_max_size = sum(clients_floors)

skyline_problem_dynamic_resolution(clients_number,
        clients_floors,client_gains,15)


In [ ]:
clients_number = 5
clients_floors = [7 6 14 7 9]
clients_gains = [1 2 5 6 11]
building_max_size = sum(clients_floors)

skyline_problem_dynamic_resolution(clients_number,
        clients_floors,client_gains,15)